In [15]:
import pandas as pd
import geopandas as gpd
import folium
from gpx_analysis import read_simple_gpx, analyze_steps, points_to_segments_lonlat, aggregate_by_hazard
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
hazard_colors = {
    "steep_climb": "#012C22",
    "climb": "#2D9966",
    "flat": "#31D492",
    "light_descent": "#fee08b",
    "steep_descent": "#f46d43",
    "ultra_steep_descent": "#9F0712",
    "turn_on_descent": "#4F39F6",
    "turn_on_steep_descent": "#8A0194"
}

In [17]:
paths = ["gpx_data\\onthegomap-16.4-mi-route (1).gpx"]

In [22]:
dfs = []
for path in paths:
    points = read_simple_gpx(path, reverse=True)
    df = analyze_steps(points, rolling_window=3)
    df['name'] = path.split(".gpx")[0]
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)

In [28]:
aggregate_by_hazard(df, column='step_dist_f')

,hazard,step_dist_f,percent
0,flat,53067.0,61.0
1,climb,15629.0,18.0
2,light_descent,8507.0,10.0
3,steep_descent,5698.0,7.0
4,steep_climb,1467.0,2.0
5,turn_on_descent,1122.0,1.0
6,turn_on_steep_descent,900.0,1.0
7,TOTAL,86390.0,100.0


In [29]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326")
gdf_segments = points_to_segments_lonlat(gdf)

In [30]:
# gdf_segments["gmaps_link"] = google_maps_link(gdf_segments['lat'], gdf_segments['lon'])
gdf_segments["gmaps_popup"] = (
    '<a href="https://www.google.com/maps?q='
    + gdf_segments["lat"].astype(str)
    + ','
    + gdf_segments["lon"].astype(str)
    + '" target="_blank">📍 Open in Google Maps</a>'
)



In [31]:
m = gdf_segments.explore(
    column='hazard',
    tooltip=[
        'hazard'
    ],
    popup=[
        'hazard',
        'step_turn',
        'step_grade',
        'avg_step_grade',
        'avg_bearing_change',
        'gmaps_popup'
    ],
    categorical=True,
    cmap=list(hazard_colors.values()),
    categories=list(hazard_colors.keys()),
    legend=True,
    style_kwds={"weight": 6},
    escape=False
)
gdf.iloc[[0]].explore(m=m,  marker_type="marker",
    tooltip=False,
    marker_kwds={
        "icon": folium.Icon(
            icon="arrow-right",
            prefix="fa",   # Font Awesome
            color="green"
        )
    }
)
gdf.iloc[[-1]].explore(m=m,  marker_type="marker",
    tooltip=False,
    marker_kwds={
        "icon": folium.Icon(
            icon="stop",
            prefix="fa",   # Font Awesome
            color="red"
        )
    }
)
m


In [10]:
from gpx_analysis import stop_signs_on_segments
all_stops = stop_signs_on_segments(gdf_segments, network_type="drive", corridor_m=6, segment_buffer_m=6)

In [11]:
lights = all_stops[all_stops["highway"] == "traffic_signals"]
stops  = all_stops[all_stops["highway"] == "stop"]

In [12]:
if 'description' in lights.columns:
    lights = lights[lights['description'] != 'flashing yellow light']

In [13]:
m = gdf_segments.explore(
    column='hazard',
    tooltip=[
        'hazard'
    ],
    popup=[
        'hazard',
        'step_turn',
        'step_grade',
        'avg_step_grade',
        'avg_bearing_change',
        'gmaps_popup'
    ],
    categorical=True,
    cmap=list(hazard_colors.values()),
    categories=list(hazard_colors.keys()),
    legend=True,
    style_kwds={"weight": 6},
    escape=False
)
gdf.iloc[[0]].explore(m=m,  marker_type="marker",
    tooltip=False,
    marker_kwds={
        "icon": folium.Icon(
            icon="arrow-right",
            prefix="fa",   # Font Awesome
            color="green"
        )
    }
)
gdf.iloc[[-1]].explore(m=m,  marker_type="marker",
    tooltip=False,
    marker_kwds={
        "icon": folium.Icon(
            icon="stop",
            prefix="fa",   # Font Awesome
            color="red"
        )
    }
)
gdf.iloc[[gdf['elevation_f'].argmax()]].explore(m=m,  marker_type="marker",
    tooltip=['elevation_f', 'elevation_m'],
    marker_kwds={
        "icon": folium.Icon(
            color="black",icon="mountain", prefix='fa'
        )
    })
stops.explore(
    m=m,
    tooltip=["highway", "street_count"],
    marker_kwds={"radius": 4, "color": "red"},
)
lights.explore(
    m=m,
    tooltip=["highway"],
    marker_kwds={"radius": 6},
)
m
